In [ ]:
# Copyright 2024 Wookjae
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Gemini 1.5 Flash 비용 계산하기


<table align="left">
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fgetting-started%2Fintro_gemini_1_5_flash.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/getting-started/intro_gemini_1_5_flash.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_1_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_1_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_1_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/53/X_logo_2023_original.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_1_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_1_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>            


| | |
|-|-|
| Author(s) |  [Wookjae Maeng](https://github.com/sociengineer)|

## 개요

Gemini의 경우 [Google AI Studio](https://ai.google.dev/pricing?_gl=1*1l09a21*_up*MQ..*_ga*MTUxNTcxNzY5NS4xNzM1MjAyNjg1*_ga_P1DBVKWT6V*MTczNTIwMjY4NS4xLjAuMTczNTIwMjczMy4wLjAuNzY5MDY2MDAx#1_5flash)와 [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/pricing)에서 과금하는 체계가 다르다. Google AI Studio에서는 토큰당 과금을 하는 반면, Vertex AI에서는 글자수에 따라 과금을 한다. 이에 따라 과금 방식이 헷갈리는 사용자를 위해서 Gemini 모델에 입력으로 넣는 특정 프롬프트와 출력의 결과를 바탕으로 금액을 계산하는 코드 샘플이다.

## 시작하기

### Python용 Vertex AI SDK 설치하기


In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.3 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Google Cloud 프로젝트 정보 설정과 Vertex AI SDK 초기화

Vertex AI를 사용하려면 기존 Google Cloud 프로젝트가 있어야 하고 [Vertex AI API를 사용 설정해야 합니다](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

[프로젝트와 개발 환경 설정](https://cloud.google.com/vertex-ai/docs/start/cloud-environment) 대해 자세히 알아보세요.

In [ ]:
# 프로젝트 ID를 제공하지 않으면 환경 변수를 사용하세요
import os

import vertexai

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=LOCATION)

### 라이브러리 가져오기


In [ ]:
import IPython.display
from vertexai.generative_models import (
    GenerativeModel,
    Part,
)

### Gemini 1.5 Flash 모델 가져오기

[Vertex AI의 모든 Gemini 모델](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models)에 대해 자세히 알아보려면 여기를 클릭하세요.

In [ ]:
MODEL_ID = "gemini-1.5-flash-002"  # @param {type:"string"}
# MODEL_ID = "gemini-1.5-pro-002"

model = GenerativeModel(MODEL_ID)

### Vertex AI SDK 기본 사용법

아래 내용은 Vertex AI SDK를 사용하여 Gemini 1.5 Flash 모델을 프롬프트하는 방법을 보여주는 간단한 예시입니다. [Gemini API 파라미터](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini#gemini-pro) 대해 자세히 알아보세요.

In [ ]:
prompt = """
응답을 한국어로 작성하세요.

Gemini 2.0을 홍보하기 위해 매력적인 마케팅 문구를 만드세요. Gemini 2.0의 다양한 측면을 강조하고 다양한 잠재 사용자를 타겟으로 하는 여러 가지 옵션을 생성하세요. 간결하고 유익한 것부터 매력적이고 열정적인 것까지 다양한 어조와 스타일을 탐구하세요.

각 마케팅 카피 옵션에 대해 다음을 제공하세요.

1. **헤드라인:** 최소 3개의 고유한 헤드라인 옵션을 제공합니다.
2. **본문:** 헤드라인에 해당하는 최소 3개의 다른 본문 변형을 제공합니다.
3. **행동 유도:** 명확하고 간결한 행동 유도를 포함합니다.

또한 소셜 미디어 게시물이나 광고와 같은 짧은 형식의 홍보 자료에 사용하기에 적합한 Gemini 2.0의 주요 기능 및 이점에 대한 간략한 요약을 제공하세요. Gemini 2.0을 경쟁 제품과 차별화하는 요소와 사용자를 위해 어떤 문제를 해결하는지 고려하세요. Gemini 2.0에 대한 흥분과 관심을 불러일으키는 데 집중하세요.

예:

**헤드라인 옵션:**
* Gemini 2.0으로 미래를 경험하세요
* Gemini 2.0: 잠재력을 발휘하세요
* Gemini 2.0으로 작업 흐름을 개선하세요

**본문 옵션:**
* (헤드라인 1에 해당) 창의적인 프로세스를 혁신하도록 설계된 최첨단 AI 모델인 Gemini 2.0으로 내일을 시작하세요. 혁신적인 아이디어 생성부터 매력적인 콘텐츠 제작에 이르기까지 Gemini 2.0을 통해 그 어느 때보다 더 많은 것을 성취할 수 있습니다.
* (헤드라인 2에 해당) Gemini 2.0을 사용하여 창의적인 노력 내에서 아직 개발되지 않은 잠재력을 발견하세요. 이 고급 AI 모델은 잠재력을 최대한 발휘하고 비전을 실현하는 데 필요한 도구와 리소스를 제공합니다.
* (헤드라인 3에 해당) Gemini 2.0으로 작업 흐름을 간소화하고 생산성을 높이세요. 이 지능형 AI 비서는 기존 도구에 원 seamlessly게 통합되어 작업을 자동화하고 가장 중요한 작업에 집중할 수 있도록 시간을 확보합니다.

**행동 유도:** 지금 바로 웹사이트를 방문하여 Gemini 2.0의 위력을 경험하고 자세히 알아보세요!

**주요 기능 및 이점 요약:** Gemini 2.0은 탁월한 창의적 기능, 원활한 워크플로우 통합 및 사용자 친화적인 인터페이스를 제공하는 차세대 AI입니다. Gemini 2.0으로 아이디어를 현실로 바꾸세요.
"""

# 모델에 전송할 내용 설정
contents = [prompt]

# 입력 토큰 수와 과금 글자 수 계산
number_of_input_tokens = model.count_tokens(contents)
# print(number_of_input_tokens)

# 모델에 콘텐츠 생성을 요청
response = model.generate_content(
    contents
)

# Gemini 응답
print(f"\n응답:\n{response.text}")

# 출력 토큰 수와 과금 글자 수 계산
number_of_output_tokens = model.count_tokens(response.text)
# print(number_of_output_tokens)


응답:
## Gemini 2.0 마케팅 문구

**옵션 1:  미래 지향적 & 전문가 타겟**

**1. 헤드라인 옵션:**
* Gemini 2.0: 차세대 AI, 새로운 가능성
* 미래의 업무 방식을 재정의하다: Gemini 2.0
* 지능형 자동화의 혁신: Gemini 2.0


**2. 본문 옵션:**
* (헤드라인 1에 해당)  Gemini 2.0은 최첨단 AI 기술을 통해 업무 방식을 혁신합니다. 복잡한 작업을 자동화하고, 데이터 분석을 간소화하며, 전략적 의사결정을 지원하여 효율성과 생산성을 극대화하세요.  경쟁 우위를 확보하고 미래를 선도하세요.
* (헤드라인 2에 해당)  시간과 자원을 절약하고, 더 중요한 업무에 집중할 수 있도록 Gemini 2.0이 도와드립니다.  자동화된 워크플로우, 지능형 분석, 그리고 사용자 친화적인 인터페이스는 귀사의 비즈니스 목표 달성에 필수적인 도구입니다.
* (헤드라인 3에 해당)  반복적인 작업을 자동화하고, 데이터 기반 의사결정을 지원하는 Gemini 2.0으로 업무 효율을 극대화하세요.  더욱 정확하고 빠른 결과를 얻고, 전략적 목표 달성에 집중할 수 있습니다.


**3. 행동 유도:** 지금 바로 데모를 신청하고 Gemini 2.0의 잠재력을 확인하세요!


**옵션 2:  창의적 & 개인 사용자 타겟**

**1. 헤드라인 옵션:**
* Gemini 2.0: 당신의 아이디어를 현실로 만들어 보세요
* 상상의 한계를 뛰어넘다: Gemini 2.0
* 창의적인 잠재력을 깨우다: Gemini 2.0


**2. 본문 옵션:**
* (헤드라인 1에 해당)  글쓰기, 그림 그리기, 음악 작곡 등 어떤 창작 활동에도 Gemini 2.0이 당신의 뮤즈가 되어줄 것입니다.  새로운 아이디어를 탐구하고, 혁신적인 콘텐츠를 제작하여 당신의 창의성을 마음껏 발휘하세요.
* (헤드라인 2에 해당)  단순한 도구를 넘어, Gemini 2.0은 당신의 영감의 파트너입니다.  상상력의 한계를 뛰어넘고, 전에 없던 새로운 경험을

In [ ]:
## Vertex AI의 Gemini 가격 계산
## 1. 입력 프롬프트 과금 글자 수: number_of_input_tokens.total_billable_characters
## 1-1. 입력 텍스트 요금 128,000 토큰 이하 $0.00001875 / 1k characters, 이상 $0.0000375 / 1k characters
input_price = 0.0

if number_of_input_tokens.total_tokens <= 128000:
  input_price = number_of_input_tokens.total_billable_characters / 1000 * 0.00001875
else:
  input_price = number_of_input_tokens.total_billable_characters / 1000 * 0.0000375


## 2. 출력 결과 과금 글자 수: number_of_output_tokens.total_billable_characters
## 2-2. 출력 텍스트 요금 128,000 토큰 이하 $0.000075 / 1k characters, 이상 $0.00015 / 1k characters

output_price = 0.0

if number_of_output_tokens.total_tokens <= 128000:
  output_price = number_of_output_tokens.total_billable_characters / 1000 * 0.000075
else:
  output_price = number_of_output_tokens.total_billable_characters / 1000 * 0.00015

total_price = input_price + output_price

In [ ]:
import requests

def get_exchange_rate(base_currency, target_currency):
  """Fetches the latest exchange rate from the Frankfurter API.

  Args:
    base_currency: The base currency (e.g., 'USD').
    target_currency: The target currency (e.g., 'KRW').

  Returns:
    The exchange rate as a float, or None if the request fails.
  """
  url = f"https://api.frankfurter.app/latest?from={base_currency}&to={target_currency}"
  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes
    data = response.json()
    return data['rates'][target_currency]
  except requests.exceptions.RequestException as e:
    print(f"Error fetching exchange rate: {e}")
    return None

if __name__ == "__main__":
  print("입력한 프롬프트에 대한 Gemini 1.5 Flash 002 가격")
  print('$'+str(total_price))
  usd_to_krw = get_exchange_rate('USD', 'KRW')
  if usd_to_krw:
    # print(f"1 USD is equal to {usd_to_krw} KRW")
    print('￦'+str(total_price * usd_to_krw))

입력한 프롬프트에 대한 Gemini 1.5 Flash 002 가격(USD): $0.00012466875
0.183724336875
